In [2]:
import numpy as np
from scipy.io import loadmat
from glob import glob
import os.path as osp

In [ ]:
file_path = "/home/haimingzhang/Research/Face/FaceFusionFormer/data/id00002/obama_weekly_003/deep3dface/000879.mat"
face_3d_params = loadmat(file_path) # dict type

exp = face_3d_params['id']
print(exp.shape)
print(exp.min(), exp.max())
print(exp)
# np.save("./exp.npy", exp)

In [9]:

def load_sequence_params(dir_path, key="id"):
    all_mat_files = sorted(glob(osp.join(dir_path, "deep3dface", "*.mat")))
    
    params = []
    for file in all_mat_files:
        face_3d_params = loadmat(file) # dict type
        data = face_3d_params[key]
        params.append(data)
    params = np.concatenate(params, axis=0)
    return params

params = load_sequence_params("/home/haimingzhang/Research/Face/FaceFusionFormer/data/id00002/obama_weekly_003")
print(params.shape)

params_diff = params[1:, :] - params[:-1, :]
print(params_diff.max())



(7243, 80)
0.8412685


## Explore the BFM model processed by MSRA 

In [4]:
import os
import random

model = loadmat(os.path.join("/data/zhanghm/3DMM/BFM", "BFM_model_front.mat"))
mean_shape = model['meanshape'].astype(np.float32) # [3*N,1]
id_base = model['idBase'].astype(np.float32) # [3*N,80]
exp_base = model['exBase'].astype(np.float32) # [3*N,64]
key_point = np.squeeze(model['keypoints']).astype(np.int64) - 1
print(key_point.dtype, key_point.shape)
print(key_point)

mean_shape = mean_shape.reshape(-1, 3)
print(mean_shape.shape)

index = np.arange(0, 35709, 6)
print(index.shape)

index = np.random.choice(len(mean_shape), 5000, replace=False)

mean_shape = mean_shape[index, :]
print(mean_shape.shape)

def write_obj(points, file, rgb=False):
    """Write obj file which can be opened by MeshLab

    Args:
        points (np.ndarray): (N, 3)
        file (str|path): save path
        rgb (bool, optional): including rgb information. Defaults to False.
    """
    fout = open(file, 'w')
    for i in range(points.shape[0]):
        if not rgb:
            fout.write('v %f %f %f %d %d %d\n' % (
                points[i, 0], points[i, 1], points[i, 2], 255, 255, 0))
        else:
            fout.write('v %f %f %f %d %d %d\n' % (
                points[i, 0], points[i, 1], points[i, 2], points[i, -3] * 255, points[i, -2] * 255,
                points[i, -1] * 255))


write_obj(mean_shape, "mean_face_sample_random.obj")

int64 (68,)
[16644 16888 16467 16264 32244 32939 33375 33654 33838 34022 34312 34766
 35472 27816 27608 27208 27440 28111 28787 29177 29382 29549 30288 30454
 30662 31056 31716  8161  8177  8187  8192  6515  7243  8204  9163  9883
  2215  3886  4920  5828  4801  3640 10455 11353 12383 14066 12653 11492
  5522  6025  7495  8215  8935 10395 10795  9555  8836  8236  7636  6915
  5909  7384  8223  9064 10537  8829  8229  7629]
(35709, 3)
(5952,)
(5000, 3)


In [41]:
import open3d as o3d
pcd = o3d.io.read_point_cloud("mean_face_pcd.xyz")
point_np = np.asarray(pcd.points)
print(point_np.shape)
downpcd = pcd.voxel_down_sample(voxel_size=0.03)

point_np = np.asarray(downpcd.points)
print(point_np.shape)

write_obj(point_np, "mean_face_sampling_0.03.obj")

(35709, 3)
(5702, 3)
